In [ ]:
import json

openapi_spec = "akoya-openapi.json"
filename = "accounts_info_get"
html_filename = f"{filename}.html"
json_filename = f"{filename}.json"
endpoint_path = "/accounts-info/{version}/{providerId}"
example_json_name = "Accounts Info Example"

def get_properties_from_ref(ref, components):
    """
    Given a $ref string, extract the actual schema it points to.
    """
    schema_name = ref.replace("#/components/schemas/", "")
    return components["schemas"][schema_name]

def merge_allOf_schemas(allOf, components):
    """
    Merge schemas under "allOf" into a single schema.
    """
    merged_schema = {"properties": {}}
    for subschema in allOf:
        if "$ref" in subschema:
            subschema = get_properties_from_ref(subschema["$ref"], components)
        if "properties" in subschema:
            merged_schema["properties"].update(subschema["properties"])
    return merged_schema

def extract_properties_from_anyOf(anyOf, components, prefix=""):
    """
    Extract properties from a given "anyOf" list.
    """
    properties = []
    for subschema in anyOf:
        properties += extract_properties(subschema, components, prefix)
    return properties

def extract_properties(schema, components, prefix=""):
    """
    Recursively extract properties from a given schema.
    """
    properties = []

    # If the schema is a dictionary of properties (common in merged schemas)
    if isinstance(schema, dict) and not any(key in schema for key in ["$ref", "allOf", "anyOf", "type"]):
        for prop_name, prop_schema in schema.items():
            properties += extract_properties(prop_schema, components, prop_name)
        return properties

    # If the schema is a reference, dereference it
    if "$ref" in schema:
        schema = get_properties_from_ref(schema["$ref"], components)

    # If the schema has "allOf", extract properties from each of the schemas within
    if "allOf" in schema:
        merged_schema = merge_allOf_schemas(schema["allOf"], components)
        properties += extract_properties(merged_schema, components, prefix)

    # If the schema has "anyOf", extract properties from each of the schemas within
    elif "anyOf" in schema:
        properties += extract_properties_from_anyOf(schema["anyOf"], components, prefix)

    else:
        schema_type = schema.get("type")

        # If the schema is of type object and has properties, iterate over them
        if schema.get("type") == "object" and "properties" in schema:
            properties += extract_properties(schema["properties"], components, prefix)
        elif schema.get("type") == "array" and "items" in schema:
            properties += extract_properties(schema["items"], components, prefix)
        else:
            prop_details = {
                "property_name": prefix,
                "property_type": schema.get("type", "N/A"),
                "property_description": schema.get("description", "N/A"),
                "enum": schema.get("enum", []),
                "nullable": schema.get("nullable", False)
            }
            properties.append(prop_details)

    return properties

# Load the OpenAPI JSON file
with open(openapi_spec, "r") as file:
    openapi_data = json.load(file)


response_schema = openapi_data["paths"][endpoint_path]["get"]["responses"]["200"]["content"]["application/json"]["schema"]
properties_list = extract_properties(response_schema, openapi_data["components"])

def get_example_response_for_endpoint(openapi_data, endpoint):
    """
    Retrieves an example JSON response for a given endpoint from the OpenAPI data.
    """
    try:
        # Iterate over HTTP verbs to find the correct endpoint data
        for verb, endpoint_data in openapi_data["paths"][endpoint].items():
            if "responses" in endpoint_data and "200" in endpoint_data["responses"]:
                example_response = endpoint_data["responses"]["200"]["content"]["application/json"]["examples"][example_json_name]["value"]
                return example_response
    except KeyError:
        return None

example_json = get_example_response_for_endpoint(openapi_data, endpoint_path)

def get_example_for_property(example_json, property_path):
    print(f"{property_path} example json: {example_json}")
    """
    Extracts an example value for a given property path from the example json.
    """
    parts = property_path.split('.')
    try:
        for part in parts:
            if isinstance(example_json, list):
                example_json = example_json[0]
            example_json = example_json[part]
        return example_json
    except (KeyError, TypeError):
        return None

def append_examples_to_properties(properties, example_json):
    """
    Appends an example to each property in the list of properties based on the example json.
    """
    for prop in properties:
        prop["example"] = get_example_for_property(example_json, prop["property_name"])
    return properties

# Add example data to properties
properties_with_examples = append_examples_to_properties(properties_list, example_json)

from IPython.display import display
import pandas as pd
def display_as_table(properties_list):
    """
    Display a list of properties as a table in a Jupyter notebook cell.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Display the DataFrame as a table in the Jupyter notebook cell
    display(df)


def save_to_html(properties_list, filename):
    """
    Render a list of properties as an HTML table and save to a file.
    """
    # Convert the properties list to a DataFrame
    df = pd.DataFrame(properties_list)

    # Save the DataFrame as an HTML file
    df.to_html(filename, index=False, escape=False)

# Save the properties list as an HTML file
save_to_html(properties_with_examples, html_filename)

def write_properties_to_file(properties, filename):
    """
    Writes the properties list to a file.
    """
    with open(filename, 'w') as f:
        json.dump(properties, f, indent=4)

def read_properties_from_file(filename):
    """
    Reads the properties list from a file.
    """
    with open(filename, 'r') as f:
        properties = json.load(f)
    return properties

# Testing the functions
write_properties_to_file(properties_with_examples, json_filename)
loaded_properties = read_properties_from_file(json_filename)

# display_as_table(properties_with_examples)

accountId example json: {'accounts': [{'investmentAccount': {'accountId': '839502593', 'accountType': 'College Savings', 'balanceType': 'ASSET', 'currency': {'currencyCode': 'USD'}, 'nickname': '529 for Kid'}}, {'investmentAccount': {'accountId': '5426873', 'accountNumberDisplay': '...7054', 'accountType': 'BROKERAGE', 'allowedCheckWriting': False, 'currency': {'currencyCode': 'USD'}, 'lastActivityDate': '2021-07-06T00:00:00Z', 'margin': False, 'nickname': 'Self-Directed', 'status': 'OPEN', 'transactionsIncluded': False}}, {'depositAccount': {'accountId': 'g833202fb0866d0ad83472c429', 'accountNumberDisplay': 'xxxxxxxx0071', 'accountType': 'CHECKING', 'balanceType': 'ASSET', 'currency': {'currencyCode': 'USD'}, 'description': 'Checking Plus', 'fiAttributes': [{'name': 'accountOpenedDate', 'value': '2020-04-23'}, {'name': 'interestPaidLastYear', 'value': '3.20'}], 'interestRate': 0.0125, 'interestRateAsOf': '2022-04-24T14:15:22Z', 'interestRateType': 'FIXED', 'lastActivityDate': '2022-04